In [2]:
using Gmsh
using Gridap
using GridapGmsh

In [8]:
model = GmshDiscreteModel("../data/VenturaAccelerometer/test_mesh_01.msh")

Info    : Reading '../data/VenturaAccelerometer/test_mesh_01.msh'...
Info    : 6 nodes
Info    : 2 elements
Info    : Done reading '../data/VenturaAccelerometer/test_mesh_01.msh'
Info    : Renumbering nodal model data (3 steps)
Info    : Renumbering element model data (2 steps)


UnstructuredDiscreteModel()

In [12]:
nodes = model.coordinates
print(nodes)

ErrorException: type UnstructuredDiscreteModel has no field coordinates

In [27]:
file = open("../data/VenturaAccelerometer/test_mesh_01.msh")
nodes_start = 0
nodes_end = 0
# Iterate over the lines of the file
for (i, line) in enumerate(eachline(file))    # Process the line

    switch line
        case "\$Nodes"
            nodes_start = i
        case "\$EndNodes"
            nodes_end = i
    end
    
end
print(nodes_start)
print(nodes_end)
# Close the file
close(file)

61 0.0 0.0 0.02 0.1 0.0 0.03 0.1 0.3 0.04 0.0 0.3 0.05 0.0 0.15 0.06 0.1 0.15 0.0